In [140]:
import pandas as pd
import numpy as np
import os
import re
my_dir = os.path.dirname(os.path.abspath('__file__'))
os.chdir(my_dir)

data = 'data/uncleaned data/local data/zillow_listings_pnw.csv'

In [141]:
df = pd.read_csv(data)

In [142]:
# Clean up home type
house_conditions = [
    df['type'].str.contains('condo', case=False, na=False),
    df['type'].str.contains('townhouse', case=False, na=False),
    df['type'].str.contains('manufactured', case=False, na=False)
]
house_values = ['condominium', 'townhouse', 'manufactured']
default_house_value = 'single family'
df['housing category'] = np.select(house_conditions, house_values, default_house_value)
df.drop('type', axis=1, inplace=True)

# Simplify and clean heating
heat_conditions = [
    df['heat type'].str.contains('ceiling', case=False, na=False),
    df['heat type'].str.contains('baseboard', case=False, na=False),
    df['heat type'].str.contains('ductless', case=False, na=False),
    df['heat type'].str.contains('forced air', case=False, na=False)
]
heat_values = ['ceiling', 'baseboard', 'ductless', 'forced air']
default_heat_value = 'other'
df['heat type'] = np.select(heat_conditions, heat_values, default_heat_value)
df.drop('heat type', axis=1, inplace=True)

# Convert HOA to yearly cost
def calculate_hoa(value):
    pattern = r"\d+"
    #if "monthly" in value:
    #    return int(value.replace(" monthly HOA fee", "").replace("$", "").replace(",","").replace(" monthly", "")) * 12
    #elif "semi-annually" in value:
    #    return int(value.replace(" semi-annually HOA fee", "").replace("$", "").replace(",","")) * 2
    #elif "annually" in value:
    #    return int(value.replace(" annually HOA fee", "").replace("$", "").replace(",","").replace(" annually", ""))
    #elif "quarterly" in value:
    #    return int(value.replace(" quarterly HOA fee", "").replace("$", "").replace(",","")) * 4
    #elif "Has" in value:
    #    return 0
    #else:
    #    return 0
    matches = re.findall(pattern, value)
    if matches:
        numeric_part = matches[0]
        return numeric_part
    else:
        return 0

df['HOA yearly'] = df['HOA cost'].astype(str).apply(calculate_hoa)
df.drop('HOA cost', axis=1, inplace=True)

# Convert sqft to integer type
def sqft_to_int(value):
    if not isinstance(value, str):
        return None

    if "sqft" in value:
        return int(value.replace(" sqft", "").replace(",", ""))
    elif "Acres" in value:
        # Assuming 1 acre = 43560 sqft
        #return int(float(value.replace(" Acres", "")) * 43560)
        return None
    else:
        return None

df['sqft'] = df['sqft'].apply(sqft_to_int)

# Convert price to int
df['listing price'] = df['listing price'].astype(int)

# Convert 'price per sqft' to integer
df['price per sqft'] = (df['price per sqft']
                        .str.replace("$", "")
                        .str.replace(",", "")
                        .str.replace(" price/sqft", "")
                        .str.replace("sqft", "")
                        .apply(lambda x: x.strip() if isinstance(x, str) else x)
                        .astype(int))

# Convert 'buyers fee' to float
df['buyers fee'] = df['buyers fee'].str.replace("buyers agency fee", "")
df['buyers fee'] = df['buyers fee'].str.replace("%", "")
df['buyers fee'] = df['buyers fee'].str.replace("¬†", "")
# Handling for when there is a fixed dollar value
def clean_buyers_fee(buyers_fee, price):
    if isinstance(buyers_fee, str) and buyers_fee.startswith('$'):
        # Convert string to float first, then if needed you can convert to int
        buyers_fee = float(buyers_fee.replace('$', '').replace(',', '').strip())
        buyers_fee = (buyers_fee / price) * 100
    return buyers_fee

df['buyers fee'] = df.apply(lambda row: clean_buyers_fee(row['buyers fee'], row['price per sqft']), axis=1)

#df['buyers fee'] = df['buyers fee'].astype(float) / 100

# Convert 'year built' to int
df['year built'] = df['year built'].str.replace('Built in ', "")
df['year built'] = df['year built'].astype(int)

# Drop duplicates and then drop the address column
df = df.drop_duplicates(subset = 'address')
df = df.drop('address', axis=1)

print(df)

          city state    zip  year built   
0    Hillsboro    OR  97123        2023  \
1    Hillsboro    OR  97123        2023   
2    Hillsboro    OR  97123        2023   
3    Hillsboro    OR  97123        2023   
4    Hillsboro    OR  97123        2023   
..         ...   ...    ...         ...   
983     Tacoma    WA  98405        1909   
984    Olympia    WA  98501        1946   
985    Olympia    WA  98501        1921   
986    Olympia    WA  98501        1924   
987    Seattle    WA  98133        2021   

                                      cooling type               garage type   
0                                      Central Air   1 Attached Garage space  \
1                                      Central Air  2 Attached Garage spaces   
2                  ENERGY STAR Qualified Equipment  2 Attached Garage spaces   
3                                      Central Air  2 Attached Garage spaces   
4                  ENERGY STAR Qualified Equipment           2 Garage spaces   
.. 

In [143]:
df.head()

,city,state,zip,year built,cooling type,garage type,sqft,price per sqft,buyers fee,listing price,housing category,HOA yearly
0,Hillsboro,OR,97123,2023,Central Air,1 Attached Garage space,3049.0,428,2,519900,single family,186
1,Hillsboro,OR,97123,2023,Central Air,2 Attached Garage spaces,NaN,320,2,729900,single family,91
2,Hillsboro,OR,97123,2023,ENERGY STAR Qualified Equipment,2 Attached Garage spaces,5227.0,379,2,689900,single family,147
3,Hillsboro,OR,97123,2023,Central Air,2 Attached Garage spaces,NaN,329,2,499900,townhouse,186
4,Hillsboro,OR,97123,2023,ENERGY STAR Qualified Equipment,2 Garage spaces,3049.0,312,2,599900,single family,147


In [144]:
df.to_csv('data/cleaned data/zillow_listings_combined_clean.csv', index = False)